This notebook shows a simple example of how to use some of the `flamingo` utilities to pre-process a dataset
and upload it as a W&B artifact.

Generally, this workflow will be performed in a dev environment on cluster so that the dataset files
can be saved on a shared volume. 
But this notebook can be run locally for educational purposes to illustrate the basic functions.

(1) Load and pre-process the base dataset from HuggingFace

In [ ]:
from datasets import load_dataset

base_dataset = "fka/awesome-chatgpt-prompts"
dataset = load_dataset(base_dataset, split="train")

dataset

In [ ]:
def preprocess_dataset(examples):
    texts = []
    for x in examples["prompt"]:
        texts.append(x[::-1])  # Dummy reverse the prompt
    examples["text"] = texts
    return examples


# Map some preprocessing function over the base dataset (e.g., for prompt formatting)
dataset = dataset.map(preprocess_dataset, batched=True, remove_columns=dataset.column_names)

dataset

(2) Save the dataset to disk

In [ ]:
from pathlib import Path

# Add an actual path here to where you want the data to live on shared storage
dataset_save_path = str(Path("example_dataset").absolute())

dataset.save_to_disk(dataset_save_path)

(3) Log the dataset directory as a W&B artifact for later use

In [2]:
import wandb

from flamingo.integrations.wandb import (
    ArtifactType,
    ArtifactURIScheme,
    log_directory_contents,
    log_directory_reference,
)
from flamingo.jobs.utils import FlamingoJobType

In [ ]:
with wandb.init(
    name="dataset-preprocessing-example",
    project="sfriedowitz-dev",
    entity="mozilla-ai",
    job_type=FlamingoJobType.PREPROCESSING,
):
    # Log a reference to the directory contents
    log_directory_reference(
        dir_path=dataset_save_path,
        artifact_name="example-dataset-artfact-reference",
        artifact_type=ArtifactType.DATASET,
        scheme=ArtifactURIScheme.FILE,
    )
    # Log and upload the directory contents
    log_directory_contents(
        dir_path=dataset_save_path,
        artifact_name="example-dataset-artfact-upload",
        artifact_type=ArtifactType.DATASET,
    )